**TP2 (Tracking) -Analyse vidéo**

Benzaied Saifeddine -RT5 groupe 1

<b>Object localization

In [1]:
import cv2
import numpy as np

# Chargement de la vidéo
video = cv2.VideoCapture("my_video_TPsAV.mp4")

# Vérification si la vidéo a bien été chargée
if not video.isOpened():
    print("Erreur lors de l'ouverture de la vidéo")
    exit()

print("Vidéo chargée avec succès")

Vidéo chargée avec succès


<i>Segmentation avec la méthode Adaptive Background Subtraction

In [2]:
# Coefficient d'adaptation pour la mise à jour du fond
alpha = 0.6

# Lire la première frame pour initialiser l'image de fond
ret, frame = video.read()
if not ret:
    print("Erreur : impossible de lire la première frame.")
    exit()

background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir la frame actuelle en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculer la différence absolue
    diff = cv2.absdiff(background.astype(np.uint8), gray)

    # Seuillage pour obtenir le masque binaire
    _, mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Nettoyage du masque : combinaison d'ouverture et de fermeture
    kernel = np.ones((5, 5), np.uint8)

    # Fermeture pour combler les petits trous
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Ouverture pour éliminer le bruit
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Afficher le masque segmenté
    cv2.imshow("Masque segmenté (Adaptive Background)", mask)

    # Mise à jour du fond avec la méthode Adaptive Background Subtraction
    background = alpha * gray + (1 - alpha) * background

    # Quitter si 'q' est pressé
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Fin de la vidéo ou erreur de lecture.


<i>Localisation d'objet avec la méthode Adaptive Background Subtraction

In [3]:
# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')
if not video.isOpened():
    print("Erreur : impossible de charger la vidéo.")
    exit()

# Coefficient d'adaptation pour la mise à jour du fond
alpha = 0.6

# Lire la première frame pour initialiser l'image de fond
ret, frame = video.read()
if not ret:
    print("Erreur : impossible de lire la première frame.")
    exit()

background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir la frame actuelle en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculer la différence absolue
    diff = cv2.absdiff(background.astype(np.uint8), gray)

    # Seuillage pour obtenir le masque binaire
    _, mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Nettoyage du masque : combinaison d'ouverture et de fermeture
    kernel = np.ones((5, 5), np.uint8)

    # Fermeture pour combler les petits trous
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Ouverture pour éliminer le bruit
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Trouver les contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Parcourir les contours et dessiner les boîtes englobantes
    for contour in contours:
        if cv2.contourArea(contour) > 3500:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Afficher la frame avec les boîtes englobantes
    cv2.imshow("Frame avec boîtes englobantes", frame)

    # Mise à jour du fond avec la méthode Adaptive Background Subtraction
    background = alpha * gray + (1 - alpha) * background

    # Quitter si 'q' est pressé
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Fin de la vidéo ou erreur de lecture.


<i>Localisation d'objet avec la méthode basée sur les couleurs

In [4]:
# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')
if not video.isOpened():
    print("Erreur : impossible de charger la vidéo.")
    exit()



# Définir les plages HSV pour détecter la couleur bleue
lower_blue = np.array([100, 150, 50])  # Borne inférieure (teinte, saturation, luminosité)
upper_blue = np.array([140, 255, 255])  # Borne supérieure

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir l'image en espace HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Créer un masque pour isoler les pixels bleus
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Nettoyage du masque avec des opérations morphologiques
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Combler les trous
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)   # Supprimer le bruit

    # Trouver les contours des régions bleues
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialiser une liste pour stocker les détections
    detections = []

    # Parcourir les contours et enregistrer les boîtes englobantes des objets pertinents
    for contour in contours:
        if cv2.contourArea(contour) > 7000:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            detections.append([x, y, w, h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Dessiner une boîte bleue

    

    # Afficher le masque et la frame avec suivi
    cv2.imshow("Masque Bleu", mask)
    cv2.imshow("Frame avec suivi", frame)

    # Quitter si 'q' est pressé
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

Fin de la vidéo ou erreur de lecture.


<b>Object Tracking

In [5]:
from tracker import *

# Create tracker object
tracker = EuclideanDistTracker()

<i>Suivi d'objet avec la méthode Adaptive Background Subtraction

In [6]:
# Coefficient d'adaptation pour la mise à jour du fond
alpha = 0.6

# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')
if not video.isOpened():
    print("Erreur : impossible de charger la vidéo.")
    exit()

# Lire la première frame pour initialiser l'image de fond
ret, frame = video.read()
if not ret:
    print("Erreur : impossible de lire la première frame.")
    exit()

background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Initialiser le tracker
tracker = EuclideanDistTracker()

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir la frame actuelle en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculer la différence absolue
    diff = cv2.absdiff(background.astype(np.uint8), gray)

    # Seuillage pour obtenir le masque binaire
    _, mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Nettoyage du masque : combinaison d'ouverture et de fermeture
    kernel = np.ones((5, 5), np.uint8)

    # Fermeture pour combler les petits trous
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Ouverture pour éliminer le bruit
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Trouver les contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialiser une liste pour stocker les détections
    detections = []

    # Parcourir les contours et enregistrer les boîtes englobantes des objets pertinents
    for contour in contours:
        if cv2.contourArea(contour) > 3500:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            detections.append([x, y, w, h])

    # Mettre à jour le tracker avec les nouvelles détections
    boxes_ids = tracker.update(detections)

    # Visualiser les boîtes englobantes et les IDs
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(frame, f"ID {id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Afficher la frame avec le suivi
    cv2.imshow("Frame avec suivi", frame)

    # Mise à jour du fond avec la méthode Adaptive Background Subtraction
    background = alpha * gray + (1 - alpha) * background

    # Quitter si 'q' est pressé
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

{7: (151, 494)}
{8: (137, 690)}
{8: (151, 675)}
{9: (205, 622)}
{9: (218, 608)}
{10: (92, 620)}
{10: (108, 612)}
{10: (131, 605)}
{10: (150, 612)}
{10: (167, 597)}
{15: (240, 555), 16: (250, 429), 17: (566, 344), 18: (414, 580)}
{15: (240, 555), 16: (269, 441), 17: (566, 344), 18: (414, 580)}
{18: (414, 580), 15: (240, 555), 16: (293, 435), 19: (571, 383), 20: (434, 563), 21: (266, 554), 22: (468, 490)}
{18: (414, 580), 15: (240, 555), 16: (293, 435), 19: (583, 378), 20: (434, 563), 21: (266, 554), 22: (468, 490)}
{20: (454, 558), 21: (266, 554), 22: (468, 490), 16: (293, 435), 19: (583, 378), 23: (602, 93)}
{20: (454, 558), 21: (287, 555), 22: (468, 490), 16: (293, 435), 19: (583, 378), 23: (602, 93)}
{20: (454, 558), 21: (287, 555), 22: (489, 496), 16: (293, 435), 19: (583, 378), 23: (602, 93)}
{20: (474, 565), 21: (287, 555), 22: (489, 496), 24: (326, 400)}
{20: (474, 565), 21: (308, 551), 22: (489, 496), 24: (326, 400)}
{28: (362, 491)}
{28: (362, 491), 29: (734, 291), 30: (391, 49

<i>Suivi d'objet avec la méthode basée sur les couleurs

In [7]:
# Charger la vidéo
video = cv2.VideoCapture('my_video_TPsAV.mp4')
if not video.isOpened():
    print("Erreur : impossible de charger la vidéo.")
    exit()

# Initialiser le tracker
tracker = EuclideanDistTracker()

# Définir les plages HSV pour détecter la couleur bleue
lower_blue = np.array([100, 150, 50])  # Borne inférieure (teinte, saturation, luminosité)
upper_blue = np.array([140, 255, 255])  # Borne supérieure

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir l'image en espace HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Créer un masque pour isoler les pixels bleus
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Nettoyage du masque avec des opérations morphologiques
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Combler les trous
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)   # Supprimer le bruit

    # Trouver les contours des régions bleues
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialiser une liste pour stocker les détections
    detections = []

    # Parcourir les contours et enregistrer les boîtes englobantes des objets pertinents
    for contour in contours:
        if cv2.contourArea(contour) > 7000:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            detections.append([x, y, w, h])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Mettre à jour le tracker avec les nouvelles détections
    boxes_ids = tracker.update(detections)

    # Visualiser les boîtes englobantes et les IDs
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(frame, f"ID {id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Afficher la frame avec suivi
    cv2.imshow("Frame avec suivi", frame)

    # Quitter si 'q' est pressé
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

{0: (127, 656)}
{3: (136, 604)}
{3: (140, 594)}
{3: (142, 586)}
{3: (143, 573)}
{3: (146, 563)}
{3: (149, 547)}
{5: (169, 544)}
{5: (178, 539)}
{6: (195, 507)}
{6: (202, 484)}
{6: (208, 494)}
{6: (215, 488)}
{6: (220, 487)}
{7: (234, 472)}
{7: (242, 458)}
{7: (256, 454)}
{7: (267, 442)}
{7: (282, 433)}
{7: (300, 422)}
{13: (446, 305)}
{13: (465, 293)}
{14: (506, 304)}
{14: (526, 308)}
{19: (675, 281)}
{20: (703, 251)}
{22: (735, 257)}
{26: (846, 332)}
{26: (858, 344)}
{26: (871, 353)}
{27: (792, 421), 28: (1009, 304)}
{31: (906, 412)}
{31: (909, 418)}
{32: (899, 473), 33: (1055, 311)}
{32: (899, 473), 33: (1055, 335)}
{37: (1114, 384), 38: (985, 435)}
{38: (1004, 443)}
{38: (994, 446), 39: (1166, 392)}
{39: (1172, 402), 38: (994, 446)}
{43: (1204, 425), 44: (1021, 469)}
{46: (1105, 517)}
{46: (1111, 532)}
{50: (1103, 573)}
{50: (1106, 579)}
{54: (1201, 627)}
{54: (1213, 639)}
Fin de la vidéo ou erreur de lecture.


<b>Feature-Based tracking

In [8]:
import datetime
import sys
import numpy as np
import cv2

#Let us define the video
# The video feed is read in as a VideoCapture object

cap = cv2.VideoCapture("my_video_TPsAV.mp4")
if not cap.isOpened():
    print("[ERROR] cannot open video file")
    sys.exit()

# Generate initial corners/features from the first frame
# Create a dict for the arguments of the main two methods: Shi Tomasi for feature extraction and Lucas and kanade OpticalFlow 
# set limit, minimum distance in pixels and quality of object corner to be tracked
parameters_shitomasi = dict(maxCorners=100, qualityLevel=0.3, minDistance=10)
# set min size of tracked object, e.g. 15x15px
parameter_lucas_kanade = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS |
                                                                      cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# create random colours for visualization for all 100 max corners for RGB channels
colours = np.random.randint(0, 255, size=(100, 3))

# get first video frame
ok, frame = cap.read()
if not ok:
    print("[ERROR] cannot get frame from video")
    sys.exit()
# convert to grayscale
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Use Shi-Tomasi to detect object corners / edges from initial frame
edges = cv2.goodFeaturesToTrack(frame_gray_init, mask = None, **parameters_shitomasi)

# [Debug] show amount of found edges
# max value = maxCorners see above. Reduce qualityLevel to get more hits
# print(len(edges))

# create a black canvas the size of the initial frame
canvas = np.zeros_like(frame)

# Optional recording parameter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
video_codec = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
prefix = 'recordings/'+datetime.datetime.now().strftime("%y%m%d_%H%M%S")
basename = "object_track.mp4"
video_output = cv2.VideoWriter("_".join([prefix, basename]), video_codec, fps, (frame_width, frame_height))

# loop through the remaining frames of the video
# and apply algorithm to track selected objects


while True:
    # get next frame
    ok, frame = cap.read()
    if not ok:
        print("[INFO] end of file reached")
        break
    # prepare grayscale image
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # update object corners by comparing with found edges in initial frame
    update_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None,
                                                         **parameter_lucas_kanade)
    # only update edges if algorithm successfully tracked
    new_edges = update_edges[status == 1]
    # to calculate directional flow we need to compare with previous position
    old_edges = edges[status == 1]

    for i, (new, old) in enumerate(zip(new_edges, old_edges)):
        a, b = new.ravel()
        c, d = old.ravel()

        # draw line between old and new corner point with random colour
        mask = cv2.line(canvas, (int(a), int(b)), (int(c), int(d)), colours[i].tolist(), 2)
        # draw circle around new position
        frame = cv2.circle(frame, (int(a), int(b)), 5, colours[i].tolist(), -1)

    result = cv2.add(frame, mask)
    # optional recording result/mask
    # video_output.write(result)
    cv2.imshow('Optical Flow (sparse)', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # overwrite initial frame with current before restarting the loop
    frame_gray_init = frame_gray.copy()
    # update to new edges before restarting the loop
    edges = new_edges.reshape(-1, 1, 2)


cv2.destroyAllWindows()
cap.release()

[INFO] end of file reached
